<a href="https://colab.research.google.com/github/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN/blob/testing/TESTING_AnimationKit_Rife_RealESRGAN_Upscaling_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AnimationKit AI - Upscaling & Interpolation using Real-ESRGAN+RIFE

**ALPHA 2: Released 9/4/21**

New features:

- Deflickering (P3)
- Upscaling from individual frames (P2)
- Target length (in seconds) for RIFE interpolation

**-WARNING: THIS IS THE TESTING BRANCH! Try the main branch for stability-**

Main branch can be found here: https://github.com/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN


Credits: Motion smoothing conceived from "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. This part of the script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video. 

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833). A demo can be found at https://raw.githubusercontent.com/xinntao/Real-ESRGAN

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

---

Testing notes:

Many ffmpeg examples at https://docs.google.com/document/d/12X_2YoCnPPN7B3OsgX39aYyRF8OF-TVStkFTkKhWrx4/edit

Python modules in Colab: https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec

Colab Markdown guide: https://colab.research.google.com/notebooks/markdown_guide.ipynb?hl=es-ni#scrollTo=Lhfnlq1Surtk



In [1]:
# @title AnimationKit AI is licensed under the MIT License
# @markdown Practical-RIFE, Real-ESRGAN, and any other external tools belong to their respective authors and may vary in licensing agreements.

# Copyright (c) 2021 Sadnow

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [2]:
!nvidia-smi

Sat Sep 25 00:30:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mount Google Drive, Install Libraries, Load Functions

Most of the code is in here. If it looks weird, it's because half-way through development I decided to make it a Python module. It's not quite portable yet, but it's getting close!

In [3]:
#https://unix.stackexchange.com/questions/70963/difference-between-2-2-dev-null-dev-null-and-dev-null-21
import os, sys, re  #>/dev/null
from multiprocessing import Process

def dir_check(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("ERROR: "+a+" does not exist!")

def dir_make(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("Creating"+a+"...")
    !mkdir -p $a

# def need_file(a):
#   import os.path
#   from os import path
#   if not path.isfile(a):
#     print("Creating"+a+"...")
#     !mkdir -p $a



def file_check(a):
  import os.path
  from os import path
  if not path.isfile(a):
    print("ERROR: "+a+" does not exist!")


class anKit:
  #def __init__(self):
  #  pass
  import os.path
  from os import path
  #video tools
  def video_sortFrames(sourceframes, destframes):  #takes frames from input folder, moves to init_frame_storage
    dir_check(sourceframes)
    dir_make(destframes)
    %cd $sourceframes
    print("Copying frames to "+destframes+" for processing...")
    !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $destframes
    %cd $destframes
    #!find -maxdepth 1 -name '*.png' -print0 | xargs -0 'mv "$0" "${0##*_}"' 
    #!find . -type f -name "*_*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    !find . -type f -name "*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    #print("Padding filenames in "+destframes+".")
    !rename 's/\d+/sprintf("%05d",$&)/e' *  #adds padding to numbers
    print("Finished copying frames to "+destframes+".")
  def video_splitFrames(sourcefile, destframes):
    file_check(sourcefile)
    dir_make(destframes)
    !ffmpeg -y -r 1 -i $sourcefile -r 1 $destframes/frame%05d.png
  def frames2video(sourceframes,fps,outputmp4):
    dir_check(sourceframes)
    %cd $sourceframes
    #!ffmpeg -r $fps -i '%d_out.png' $outputmp4
    #!ffmpeg -y -f image2 -framerate $fps -pattern_type glob -i '*.png' $outputmp4
    #!ffmpeg -f -framerate $fps -pattern_type glob -i '*.png' -y $outputmp4
    !ffmpeg -framerate $fps -pattern_type glob -i '*.png' -y $outputmp4

  def video_runUpscale(esrgan,mpath,scale,input,output):
    dir_check(input)
    dir_make(output)
    %cd $esrgan
    #print(mpath)
    !python inference_realesrgan.py --model_path $mpath --netscale $scale --input $input --output $output
    #%cd $output
    #!find . -type f -name "*_*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers from filename - needed for rife-frame
    #dir_numberizeFiles(output,'.png')  def video_compress(input,effects,quality,output):  #needs portable
    #!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $target_output_fullpath
  def RIFE_video(fps,exp,input,scale,output):
    %cd /content/Practical-RIFE
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --video=$input --scale $scale --output=$output
  def RIFE_frames(fps,exp,input,scale,output):  #not currently working right
    %cd /content/Practical-RIFE
    input = input + '/'
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --img=$input --scale $scale --output=$output
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = int(a_string)
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = target_fps * target_length_seconds
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x
  #installation for Colab (untested on other platforms)
  #
  #
  def install_ESRGAN():
    print("Installing libraries for Real-ESRGAN upscaling.")
    !git clone https://github.com/xinntao/Real-ESRGAN.git &> /dev/null
    %cd Real-ESRGAN
    !pip -q install basicsr
    !pip -q install facexlib
    !pip -q install gfpgan
    !pip -q install -r requirements.txt   &> /dev/null
    !python setup.py develop              &> /dev/null
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models              &> /dev/null
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models   &> /dev/null
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models              &> /dev/null
    print("Finished Installing libraries for Real-ESRGAN upscaling.")
    #
  def install_RIFE():
    %cd /content/
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/Practical-RIFE Practical-RIFE &> /dev/null
    !gdown --id 1O5KfS3KzZCY3imeCr2LCsntLhutKuAqj &> /dev/null
    #%cd /content/Practical-RIFE
    !7z e RIFE_trained_model_v3.8.zip &> /dev/null
    #!7z e /content/Practical-RIFE/RIFE_trained_model_v3.8.zip
    #!7z e /content/RIFE_trained_model_v3.8.zip
    !mkdir /content/Practical-RIFE/train_log  &> /dev/null
    !mv *.py /content/Practical-RIFE/train_log/ &> /dev/null
    !mv *.pkl /content/Practical-RIFE/train_log/  &> /dev/null
    %cd /content/Practical-RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc   &> /dev/null  #useless - mp4 demo 
    !pip3 install -r requirements.txt &> /dev/null
    print("Finsihed Installing libraries for RIFE motion smoothing.")
    #  
    %cd /content/
  def depcrecated_installOldRIFE():
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
    !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
    !7z e RIFE_trained_model_HDv2.zip
    !mkdir /content/RIFE/train_log
    !mv *.pkl /content/RIFE/train_log/
    %cd /content/RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
    !pip3 install -r requirements.txt
    print("Done.")
    print("Finsihed Installing libraries for RIFE motion smoothing.")
  %cd /content/
  
  def install_3dinpainting(): #untested as a function
    !pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
    !pip3 install opencv-python==4.2.0.32
    !pip3 install vispy==0.6.4
    !pip3 install moviepy==1.0.2
    !pip3 install transforms3d==0.3.1
    !pip3 install networkx==2.3
    !sudo apt install sed
    %cd /content/
    !git clone https://github.com/sadnow/3d-photo-inpainting.git
    %cd 3d-photo-inpainting
    !sh download.sh
    !sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

  def run_3dinpainting(): #untested as a function
    !mkdir image
    !mkdir video
    %cd image
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    %cd ..
    !python main.py --config argument.yml
###


###############################################################

#
#If RIFE (P2) stops for no reason, go to Runtime>Change Runtime type and set the notebook to "high memory"

#Params
mount_google_drive = True #@param {type:"boolean"}

#Mount Google Drive
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

#install dependencies
anKit.install_ESRGAN()
anKit.install_RIFE()
####################################################

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Installing libraries for Real-ESRGAN upscaling.
/content/Real-ESRGAN
Finished Installing libraries for Real-ESRGAN upscaling.
/content
Installing libraries for RIFE motion smoothing.
/content/Practical-RIFE
Finsihed Installing libraries for RIFE motion smoothing.
/content


# Real-ESRGAN Video Upscaling
Planned additions: Better flickering, faces option, tiles option

---

# Motion interpolation (Practical-RIFE Smoothing)

New: Type in the number of seconds you want your clip to be!

Lower the `target_scale_RIFE` to speed up RIFE (at the cost of resolution). A value of `0.5` would make a 4k video 2k.

*Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)*

---

# P3: Deflickering & x265 Compression (Experimental, Very Fast)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss. Deflickering is good for deflickering stopmotions, VQGAN_CLIP animations, etc.

Also, too high of compressions might require a decent local GPU to view. Will add h264 toggle in the future~

In [4]:
#@markdown # AnimationKit AI
#@markdown >*This cell combines upscaling, RIFE smoothing, and everything else*
#@markdown ---
#@markdown ##Input settings
##@markdown >For importing mp4 files:
import_mp4 = False #@param {type:"boolean"}
imported_mp4='' #@param {type:"string"}
##@markdown `
##@markdown >For importing individual frames:
import_frame_folder='/content/drive/MyDrive/VQLIPSE/images_out/birdsflapping' #@param {type:"string"}
#@markdown Note: *`import_frame_folder` is for importing individual frames from your projects. Will be ignored if `import_mp4` is checked.*
#---------------------------------------------------------------#
#@markdown ---
#@markdown ##Output settings
output_folder = '/content/drive/MyDrive/pytti_test/videos/' #@param {type:"string"}
output_filename = 'birdsflapping_yes-removestatic.mp4' #@param{type:"string"}
target_output_fullpath = output_folder + output_filename
target_fps=60#@param {type:"integer"}
target_length_seconds=5#@param {type:"integer"}
remove_static_frames = False #@param {type:"boolean"}
upscale_model_path='/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth' #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
constant_quality=25#@param {type:"slider", min:20, max:50, step:1}
face_enhance_gfpgan=True #@param{type:"boolean"}
#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality

#@markdown `remove_static_frames` will skip duplicate frames during RIFE. If you your video seems choppy, try playing with this setting and/or `target_scale_RIFE` below.
#---------------------------------------------------------------#
#@markdown ---
#@markdown ##Speed Optimization
#input_path='/content/drive/MyDrive/pytti_test/videos/zelda4.mp4' #@param {type:"string"}
target_scale_RIFE='1.0'#@param ['0.25','0.5','1.0','2.0','4.0']{type:"string"}
half_precision_realesrgan=False #@param{type:"boolean"}
#length_multiplier=3#@param {type:"integer"}
#@markdown *These options can speed up processing at the costs of quality.*
#@markdown If you're seeing weird warping in your outputs, try increasing `target_scale_RIFE`.
#@markdown 
#---------------------------------------------------------------#
#@markdown ---
#@markdown #Visual Effects & Compression
#compress_path='/content/drive/MyDrive/pytti_test/videos/zelda4_16X_180fps.mp4' #@param {type:"string"}
#outputStr = '_tblend2.mp4' #@param {type:"string"}
enable_visual_effects = False #@param {type:"boolean"}
#deflicker_on = False #@param {type:"boolean"}
#deflicker_avg_frames=76#@param {type:"slider", min:2, max:129, step:1}
#mpdecimate_on = False #@param {type:"boolean"}
#minterpolate_on = False #@param {type:"boolean"}
#minterpolate_fps=60#@param {type:"integer"}
tblend_on = False #@param {type:"boolean"}
#tmix_on = True #@param {type:"boolean"}
tblend_framestep_value = "2" #@param {type:"string"}
##@markdown Set `deflicker_avg_frames` filter size in frames. FFmpeg's default is 5. I haven't tested too much but I would try 100.

##@markdown ---
#@markdown `Tblend` divides your `target_fps` but blends frames (deflickering effect). A value of `2` with a `target_fps` of `120` would output a `60 fps` video.
#@markdown Disable visual_effects if you don't have any visual effects checked-

#---------------------------------------------------------------#
#@markdown ---
#@markdown ##Master Options
#@markdown If you want to skip certain processes, this is the master toggle. All related settings above will be ignored, so no need to change them.
skip_cleanup = False #@param{type:"boolean"}
skip_upscaling = False #@param{type:"boolean"}
skip_RIFEsmoothing = False #@param{type:"boolean"}
#---------------------------------------------------------------#



###################################################################
#Conditional passing of flags

if enable_visual_effects:
  visual_effects = '-vf'
  #if minterpolate_on:
  #  visual_effects = visual_effects + ' minterpolate=fps=' + str(minterpolate_fps) + ':mi_mode=blend'
  # if deflicker_on:
  #   visual_effects = visual_effects + ' deflicker=s=' + str(deflicker_avg_frames) + ':m=am'
  #if mpdecimate_on:
    #visual_effects = visual_effects + ' mpdecimate'
  if tblend_on:
    visual_effects = visual_effects + ' tblend=all_mode=average,framestep=' + tblend_framestep_value
#  if tmix_on:
#    visual_effects = visual_effects + ' tmix=frames=5:weights=\"1 1 1 1 1\",select=\'not(mod(n\,5))\''
#  if deblock_toggle:
#    x265_params = '-x265-params deblock=-3,-3 '
else:
  visual_effects = ''

if upscale_model_path == '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth':
  upscale_value="2"
else:
  upscale_value="4"

if half_precision_realesrgan:
  upscale_value = upscale_value + " --half"
if face_enhance_gfpgan: 
  upscale_value = upscale_value + " --face_enhance"

if remove_static_frames:
  target_scale_RIFE = target_scale_RIFE + ' --skip'


###############################################################
#Important paths
init_frame_storage = "/content/frames_storage/init_frame_storage"
esrgan_inference_path = "/content/Real-ESRGAN/"
processed_frame_storage = "/content/frames_storage/upscaled_frame_storage"
###############################################################
%cd /content/
#cleanup
if not skip_cleanup:
  print("Cleaning up from last run...")
  !rm -rf "/content/Real-ESRGAN/results"
  !rm -rf $init_frame_storage
  !rm -rf $processed_frame_storage

###############################################################
#P1 REAL-ESRGAN UPSCALING
print("------------------------------------")
print("Beginning Real-ESRGAN Upscaling phase...")

if import_mp4:
  anKit.video_splitFrames(imported_mp4,init_frame_storage) #split mp4 into frames
  anKit.video_sortFrames(init_frame_storage,init_frame_storage)
  prior_path = imported_mp4
else:
  anKit.video_sortFrames(import_frame_folder,init_frame_storage)  #duplicates frames to init_frame_storage
  #not necessary if import_mp4 checked bcuz video_splitFrames() saves as %d
  prior_path = target_output_fullpath

%cd /content/

#-----------------------------
if skip_upscaling:
  print("NOTICE: skip_upscaling is checked. Skipping Real-ESRGAN upscaling...")
  next_path = prior_path
else:
  next_path = prior_path + '_upscaled.mp4'
  anKit.video_runUpscale(esrgan_inference_path,upscale_model_path,upscale_value,init_frame_storage,processed_frame_storage)
#----------------------------
print("NOTICE: Running frames2video")
anKit.frames2video(processed_frame_storage,target_fps,prior_path)
#frames2video wont be necessary once I get rife to input bulk individual frames
print("NOTICE: Finished running frames2video")
#END OF UPSCALING PHASE
print("End of upscaling phase.")

###############################################################################
#P2 RIFE MOTION SMOOTHING
print("------------------------------------")
print("Beginning RIFE motion smoothing phase...")

%cd /content/Practical-RIFE/

measured_fps = anKit.detect_fps(prior_path)
print("Detected average FPS of",prior_path,"is",measured_fps)
measured_duration = anKit.detect_duration(prior_path)
print("Detected duration INTEGER (in seconds) is",measured_duration)
if measured_duration < 1:
  measured_duration = 0.4

exp_value = anKit.exp_calc()

print("Warning: Target duration currently rounds to the closest --exp RIFE can handle. ")

if (exp_value < 0.5):
  skip_RIFEsmoothing = True
  print("NOTICE: Your target_fps doesn't necessitate RIFE motion smoothing. Skipping RIFE...")

if skip_RIFEsmoothing:
  next_path = prior_path
  print("NOTE: Skipping RIFE motion smoothing...")
else:
  next_path = prior_path + '_smooth.mp4'
  anKit.RIFE_video(target_fps,exp_value,prior_path,target_scale_RIFE,next_path)

#target_output_fullpath_upscaled_smooth = target_output_fullpath_upscaled + '_smooth.mp4'

#!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $target_output_fullpath


#anKit.RIFE_frames(target_fps,exp_value,processed_frame_storage,target_scale_RIFE,target_output_fullpath_upscaled_smooth)
#END OF MOTION SMOOTHING PHASE
print("End of RIFE interpolation phase.")

#########################################################################
#h265 compression
print("------------------------------------")
print("Beginning h265 compression phase...")

#def video_compress(next_path,visual_effects,constant_quality,target_output_fullpath):
#  !ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $target_output_fullpath


%cd /content/  
!ffmpeg -y -i $next_path $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $target_output_fullpath
print("End of h265 compression phase.")
print("------------------------------")
print("Finished! Your final file is saved as",target_output_fullpath)

/content
Cleaning up from last run...
------------------------------------
Beginning Real-ESRGAN Upscaling phase...
Creating/content/frames_storage/init_frame_storage...
/content/drive/MyDrive/VQLIPSE/images_out/birdsflapping
Copying frames to /content/frames_storage/init_frame_storage for processing...
/content/frames_storage/init_frame_storage
Finished copying frames to /content/frames_storage/init_frame_storage.
/content
Creating/content/frames_storage/upscaled_frame_storage...
/content/Real-ESRGAN
Testing 0 00001
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Testing 1 00002
Testing 2 00003
Testing 3 00004
Testing 4 00005
Testing 5